### 参考
[safwankdb/ResNet34-TF2](https://github.com/safwankdb/ResNet34-TF2)

In [28]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
mirrored_strategy = tf.distribute.MirroredStrategy()

Num GPUs Available:  8
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


In [29]:
# from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [40]:
import os
path_data = '../data/dataset/'
# path_data = '../data/augumented_dataset/'
path_train = path_data+'train/'
path_val = path_data+'test/'

num_l_train = len(os.listdir(path_train+'left'))
num_r_train = len(os.listdir(path_train+'right'))

num_l_val = len(os.listdir(path_val+'right'))
num_r_val = len(os.listdir(path_val+'left'))

total_train = num_l_train + num_r_train
total_val = num_l_val + num_r_val

In [41]:
batch_size = 32
# batch_size = 256
epochs = 30
IMG_HEIGHT = 32
IMG_WIDTH = 32

In [42]:
train_image_generator = ImageDataGenerator(rescale=1./255) # 学習データのジェネレータ
validation_image_generator = ImageDataGenerator(rescale=1./255) # 検証データのジェネレータ

In [43]:
train_data_gen = train_image_generator.flow_from_directory(directory=path_train,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 2286 images belonging to 2 classes.


In [44]:
val_data_gen = validation_image_generator.flow_from_directory(directory=path_val,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 764 images belonging to 2 classes.


In [45]:
from tensorflow import keras
from tensorflow.keras.activations import relu
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras import layers as Layers

In [46]:
class ResBlock(Model):
    def __init__(self, channels, stride=1):
        super(ResBlock, self).__init__(name='ResBlock')
        self.flag = (stride != 1)
        self.conv1 = Conv2D(channels, 3, stride, padding='same')
        self.bn1 = BatchNormalization()
        self.conv2 = Conv2D(channels, 3, padding='same')
        self.bn2 = BatchNormalization()
        self.relu = ReLU()
        if self.flag:
            self.bn3 = BatchNormalization()
            self.conv3 = Conv2D(channels, 1, stride)

    def call(self, x):
        x1 = self.conv1(x)
        x1 = self.bn1(x1)
        x1 = self.relu(x1)
        x1 = self.conv2(x1)
        x1 = self.bn2(x1)
        if self.flag:
            x = self.conv3(x)
            x = self.bn3(x)
        x1 = Layers.add([x, x1])
        x1 = self.relu(x1)
        return x1

In [47]:
class ResNet34(Model):
    def __init__(self):
        super(ResNet34, self).__init__(name='ResNet34')
        self.conv1 = Conv2D(64, 7, 2, padding='same')
        self.bn = BatchNormalization()
        self.relu = ReLU()
        self.mp1 = MaxPooling2D(3, 2)

        self.conv2_1 = ResBlock(64)
        self.conv2_2 = ResBlock(64)
        self.conv2_3 = ResBlock(64)

        self.conv3_1 = ResBlock(128, 2)
        self.conv3_2 = ResBlock(128)
        self.conv3_3 = ResBlock(128)
        self.conv3_4 = ResBlock(128)

        self.conv4_1 = ResBlock(256, 2)
        self.conv4_2 = ResBlock(256)
        self.conv4_3 = ResBlock(256)
        self.conv4_4 = ResBlock(256)
        self.conv4_5 = ResBlock(256)
        self.conv4_6 = ResBlock(256)

        self.conv5_1 = ResBlock(512, 2)
        self.conv5_2 = ResBlock(512)
        self.conv5_3 = ResBlock(512)

#         self.pool = GlobalAveragePooling2D()
#         self.fc1 = Dense(512, activation='relu')
#         self.dp1 = Dropout(0.5)
#         self.fc2 = Dense(512, activation='relu')
#         self.dp2 = Dropout(0.5)
#         self.fc3 = Dense(64)

        self.pool = GlobalAveragePooling2D()
        self.fc1 = Dense(1, activation='sigmoid')
#         self.dp1 = Dropout(0.5)
#         self.fc2 = Dense(512, activation='relu')
#         self.dp2 = Dropout(0.5)
#         self.fc3 = Dense(64)

    def call(self, x):
        x = self.conv1(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.mp1(x)

        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.conv2_3(x)

        x = self.conv3_1(x)
        x = self.conv3_2(x)
        x = self.conv3_3(x)
        x = self.conv3_4(x)

        x = self.conv4_1(x)
        x = self.conv4_2(x)
        x = self.conv4_3(x)
        x = self.conv4_4(x)
        x = self.conv4_5(x)
        x = self.conv4_6(x)

        x = self.conv5_1(x)
        x = self.conv5_2(x)
        x = self.conv5_3(x)

#         x = self.pool(x)
#         x = self.fc1(x)
#         x = self.dp1(x)
#         x = self.fc2(x)
#         x = self.dp2(x)
#         x = self.fc3(x)

        x = self.pool(x)
        x = self.fc1(x)
        return x

model = ResNet34()
model.build(input_shape=(1, 32, 32, 3))
model.summary()

Model: "ResNet34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_180 (Conv2D)          multiple                  9472      
_________________________________________________________________
batch_normalization_180 (Bat multiple                  256       
_________________________________________________________________
re_lu_85 (ReLU)              multiple                  0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 multiple                  0         
_________________________________________________________________
ResBlock (ResBlock)          multiple                  74368     
_________________________________________________________________
ResBlock (ResBlock)          multiple                  74368     
_________________________________________________________________
ResBlock (ResBlock)          multiple                  743

In [48]:
from tensorflow.keras import optimizers
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-3),
              metrics=['accuracy'])

In [49]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/30


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node ResNet34/conv2d_180/Conv2D (defined at <ipython-input-47-8957cd122a41>:44) ]] [Op:__inference_train_function_31308]

Errors may have originated from an input operation.
Input Source operations connected to node ResNet34/conv2d_180/Conv2D:
 IteratorGetNext (defined at <ipython-input-49-01c6f78f4d4f>:6)

Function call stack:
train_function


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()